In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import numpy as np
from pathlib import Path
from runner import Runner
import interest_evolution
from full_slate_q_agent import FullSlateQAgent

In [10]:
def create_dqn_agent(env, eval_mode=False, summary_writer=None):
    return FullSlateQAgent(
        observation_space=env.observation_space,
        action_space=env.action_space,
        eval_mode=eval_mode,
        summary_writer=summary_writer
    )

In [ ]:
seed = 0
np.random.seed(seed)

env_config = {
    'num_candidates': 10,
    'slate_size': 2,
    'resample_documents': True,
    'seed': seed,
}

In [ ]:
import shutil
import os
from pathlib import Path

tmp_base_dir = './logs/dqn'

# Automatically delete train and eval directories if they exist
train_log_dir = os.path.join(tmp_base_dir, 'train')
eval_log_dir = os.path.join(tmp_base_dir, 'eval')

for log_dir in [train_log_dir, eval_log_dir]:
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)

# Recreate base log directory
Path(tmp_base_dir).mkdir(parents=True, exist_ok=True)

# --- Initialize training runner ---
runner_dqn = Runner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_dqn_agent,
    env=interest_evolution.create_environment(env_config),
)

# --- Run training + evaluation ---
runner_dqn.run_training(max_training_steps=50, num_iterations=5)
runner_dqn.run_evaluation(max_eval_episodes=2)


[TRAIN][Step 60] AvgLen: 60.00 | AvgRew: 167.04 | StdRew: 0.00 | Time/Step: 0.4435


In [7]:
from random_agent import RandomAgent
import shutil
import os
from pathlib import Path

def create_random_agent(env, **kwargs):
    return RandomAgent(action_space=env.action_space)

tmp_base_dir = './logs/random'

# Automatically delete train and eval directories if they exist
train_log_dir = os.path.join(tmp_base_dir, 'train')
eval_log_dir = os.path.join(tmp_base_dir, 'eval')

for log_dir in [train_log_dir, eval_log_dir]:
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)

# Recreate base log directory
Path(tmp_base_dir).mkdir(parents=True, exist_ok=True)

# --- Initialize runner for RandomAgent ---
runner_random = Runner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_random_agent,
    env=interest_evolution.create_environment(env_config),
)

# --- Run training + evaluation ---
runner_random.run_training(max_training_steps=50, num_iterations=5)
runner_random.run_evaluation(max_eval_episodes=2)


AttributeError: module 'helper.environment' has no attribute 'Environment'